## <span style="color:#ff5f27">🧬 Train Retrieval Model </span>


## <span style="color:#ff5f27">📝 Imports </span>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import StringLookup, Normalization
import tensorflow_recommenders as tfrs
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

2024-04-21 22:55:10.031231: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 22:55:10.034162: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 22:55:10.061324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 22:55:10.061348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 22:55:10.062207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [2]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🔪 Feature Selection </span>


In [3]:
users_fg = fs.get_feature_group(
    name="users",
    version=1,
)

videos_fg = fs.get_feature_group(
    name="videos",
    version=1,
)

interactions_fg = fs.get_feature_group(
    name="interactions",
    version=1,
)

---

---

In [5]:
QUERY_FEATURES = ["user_id", "gender", "age", "country"]
CANDIDATE_FEATURES = ["video_id", "category", "views", "likes", "video_length"]

In [6]:
# Select features for training data
selected_features = interactions_fg.select(["interaction_id"])\
    .join(users_fg.select(QUERY_FEATURES), on="user_id")\
    .join(videos_fg.select(CANDIDATE_FEATURES), on="video_id")

# Uncomment this if you would like to view your selected features
# selected_features.show(5)

## <span style="color:#ff5f27">⚙️ Feature View Creation </span>


In [7]:
feature_view = fs.get_or_create_feature_view(
    name='retrieval',
    version=1,
    query=selected_features,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fv/retrieval/version/1


## <span style="color:#ff5f27">🏋️ Training Dataset </span>


In [8]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)
train_df.head(3)

Finished: Reading data from Hopsworks, using ArrowFlight (73.08s) 


,interaction_id,user_id,gender,age,country,video_id,category,views,likes,video_length
1,7575-45-0675,CV657Y,Female,46,Denmark,7MI02X,Dance,82199,50628,107
2,7107-81-7490,ML423N,Female,30,Liberia,5YC51I,Music,331845,147385,171
4,4402-90-5971,WQ755R,Male,71,Andorra,4YQ61G,Lifestyle,89101,27527,20


You will train your retrieval model with a subset of features.

For the query embedding you will use:
- `user_id`: ID of a user.
- `gender`: Gender of a user.
- `age`: age of a user.
- `country`: country if a user.

For the candidate embedding you will use:
- `video_id`: ID of a video.
- `category`: Video Category.
- `views`: Number of video views.
- `likes`: Number of video likes.
- `video_length`: Length of video.


In [9]:
def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col: df[col] for col in df})

BATCH_SIZE = 2048
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(val_df).batch(BATCH_SIZE).cache()

2024-04-21 22:59:39.055446: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-21 22:59:39.056074: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
# Query Features 
user_id_list = train_df["user_id"].unique().tolist()
countries_list = train_df["country"].unique().tolist()
gender_list = train_df["gender"].unique().tolist()

# Item Features
video_id_list = train_df["video_id"].unique().tolist()
category_list = train_df["category"].unique().tolist()

print(f"⛳️ Number of users: {len(user_id_list)}")
print(f"⛳️ Number of items: {len(video_id_list)}")

⛳️ Number of users: 24983
⛳️ Number of items: 24989


## <span style="color:#ff5f27">🏰 Two Tower Model </span>


In [11]:
EMB_DIM = 16

In [12]:
class QueryTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.emb_dim = EMB_DIM
        self.user_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=user_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # You add an additional embedding to account for unknown tokens.
                len(user_id_list) + 1,
                self.emb_dim
            )
        ])

        self.normalized_age = Normalization(axis=None)
        
        # Converts strings into integer indices (scikit-learn LabelEncoder analog)
        self.gender_tokenizer = StringLookup(
            vocabulary=gender_list,
            mask_token=None,
        )
        
        self.country_tokenizer = StringLookup(
            vocabulary=countries_list, 
            mask_token=None,
        )

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(self.emb_dim, activation="relu"),
            tf.keras.layers.Dense(self.emb_dim)
        ])

    def call(self, inputs):
        gender_embedding = tf.one_hot(
            self.gender_tokenizer(inputs["gender"]),
            len(gender_list),
        )
        
        country_embedding = tf.one_hot(
            self.country_tokenizer(inputs["country"]),
            len(countries_list),
        )
        
        concatenated_inputs = tf.concat([
            self.user_embedding(inputs["user_id"]),
            tf.reshape(self.normalized_age(inputs["age"]), (-1,1)),
            gender_embedding,
            country_embedding,
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


query_model = QueryTower()

query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[QUERY_FEATURES]
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))

<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[-0.05291146,  0.26739523, -0.18052827,  0.09963859, -0.05545553,
        -0.16253799, -0.25082752,  0.11558817,  0.19560912, -0.05760721,
        -0.06640349,  0.04214422, -0.06638975, -0.00250891,  0.24153207,
        -0.02460788]], dtype=float32)>

In [13]:
class ItemTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.emb_dim = EMB_DIM
        self.video_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=video_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # You add an additional embedding to account for unknown tokens.
                len(video_id_list) + 1,
                self.emb_dim,
            )
        ])
        
        # Converts strings into integer indices (scikit-learn LabelEncoder analog)
        self.category_tokenizer = StringLookup(
            vocabulary=category_list, 
            mask_token=None,
        )
        
        self.normalized_views = Normalization(axis=None)
        self.normalized_likes = Normalization(axis=None)
        self.normalized_video_length = Normalization(axis=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(self.emb_dim, activation="relu"),
            tf.keras.layers.Dense(self.emb_dim)
        ])

    def call(self, inputs):
        category_embedding = tf.one_hot(
            self.category_tokenizer(inputs["category"]),
            len(category_list),
        )

        concatenated_inputs = tf.concat([
            self.video_embedding(inputs["video_id"]),
            category_embedding,
            tf.reshape(self.normalized_views(inputs["views"]), (-1,1)),
            tf.reshape(self.normalized_views(inputs["likes"]), (-1,1)),
            tf.reshape(self.normalized_views(inputs["video_length"]), (-1,1)),
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs

    
item_model = ItemTower()

item_df = train_df[CANDIDATE_FEATURES]
item_df.drop_duplicates(subset="video_id", inplace=True)
item_ds = df_to_ds(item_df)

item_model(next(iter(item_ds.batch(1))))

<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[-14518.485  , -16196.728  ,   3582.8022 ,  18518.92   ,
        -18638.053  ,  -4019.3362 ,   4641.3384 ,  14198.993  ,
        -34744.555  , -21433.63   ,   8356.484  ,  -1009.62213,
         -6945.329  ,  12276.024  , -12161.892  ,  17557.12   ]],
      dtype=float32)>

In [14]:
class TwoTowerModel(tf.keras.Model):
    def __init__(self, query_model, item_model):
        super().__init__()
        self.query_model = query_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=item_ds.batch(BATCH_SIZE).map(self.item_model)
            )
        )

    def train_step(self, batch) -> tf.Tensor:
        # Set up a gradient tape to record gradients.
        with tf.GradientTape() as tape:

            # Loss computation.
            user_embeddings = self.query_model(batch)
            item_embeddings = self.item_model(batch)
            loss = self.task(
                user_embeddings, 
                item_embeddings,
                compute_metrics=False,
            )

            # Handle regularization losses as well.
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        metrics = {
            "loss": loss,
            "regularization_loss": regularization_loss,
            "total_loss": total_loss
        }

        return metrics

    def test_step(self, batch) -> tf.Tensor:
        # Loss computation.
        user_embeddings = self.query_model(batch)
        item_embeddings = self.item_model(batch)

        loss = self.task(
            user_embeddings, 
            item_embeddings,
            compute_metrics=False,
        )

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

### <span style="color:#ff5f27">🏃🏻‍♂️ Model Training </span>

You'll train our model using the AdamW optimizer, which applies weight regularization during training.

In [15]:
# Create a TwoTowerModel with the specified query_model and item_model
model = TwoTowerModel(query_model, item_model)

# Define an optimizer using AdamW with a learning rate of 0.01
optimizer = tf.keras.optimizers.AdamW(
    weight_decay=0.001, 
    learning_rate=0.01,
)

# Compile the model using the specified optimizer
model.compile(optimizer=optimizer)

In [16]:
model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=5,
)

Epoch 1/5
391/391 [==============================] - 19s 46ms/step - loss: 528366.4635 - regularization_loss: 0.0000e+00 - total_loss: 528366.4635 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 12887.0391 - val_regularization_loss: 0.0000e+00 - val_total_loss: 12887.0391
Epoch 2/5
391/391 [==============================] - 21s 53ms/step - loss: 15614.2414 - regularization_loss: 0.0000e+00 - total_loss: 15614.2414 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0

## <span style="color:#ff5f27">🗄️ Upload Model to Model Registry </span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

In [17]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [18]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function()
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {
            "user_id": instances["user_id"],
            "gender": instances["gender"],
            "age": instances["age"],
            "country": instances["country"],
            "query_emb": query_emb,
        }

# wrap query_model:   query_model -> query_model_module
query_model = QueryModelModule(model.query_model)

In [19]:
# Define the input specifications for the instances
instances_spec = {
    'user_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='user_id'),   # Specification for user IDs
    'gender': tf.TensorSpec(shape=(None,), dtype=tf.string, name='gender'),     # Specification for gender
    'country': tf.TensorSpec(shape=(None,), dtype=tf.string, name='country'),    # Specification for country
    'age': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='age'),              # Specification for age
}

# Get the concrete function for the query_model's compute_emb function using the specified input signatures
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

# Save the query_model along with the concrete function signatures
tf.saved_model.save(
    query_model,           # The model to save
    "query_model",         # Path to save the model
    signatures=signatures, # Concrete function signatures to include
)

INFO:tensorflow:Assets written to: query_model/assets
INFO:tensorflow:Assets written to: query_model/assets


In [20]:
tf.saved_model.save(
    model.item_model,    # The model to save
    "candidate_model",   # Path to save the model
)

INFO:tensorflow:Assets written to: candidate_model/assets
INFO:tensorflow:Assets written to: candidate_model/assets


In [21]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Infer input schema from data.
query_model_input_schema = Schema(query_df)

# Manually specify output schema.
query_model_output_schema = Schema([{
    "name": "query_embedding",
    "type": "float32",
    "shape": [EMB_DIM],
}])

query_model_schema = ModelSchema(
    input_schema=query_model_input_schema,
    output_schema=query_model_output_schema,
)

query_model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'user_id', 'type': 'object'},
   {'name': 'gender', 'type': 'object'},
   {'name': 'age', 'type': 'int64'},
   {'name': 'country', 'type': 'object'}]},
 'output_schema': {'tensor_schema': [{'name': 'query_embedding',
    'shape': '[16]',
    'type': 'float32'}]}}

In [22]:
# Sample a query example from the query DataFrame
query_example = query_df.sample().to_dict("records")

# Create a tensorflow model for the query_model in the Model Registry 
mr_query_model = mr.tensorflow.create_model(
    name="query_model",                                           # Name of the model
    description="Model that generates query embeddings from user features",  # Description of the model
    input_example=query_example,                                  # Example input for the model
    model_schema=query_model_schema,                              # Schema of the model
)

# Save the query_model to the Model Registry
mr_query_model.save("query_model")                                # Path to save the model

Uploading model files (2 dirs, 0 files):  17%|███████████████▏                                                                           | 1/6 [00:00<00:01,  3.17it/s]
Uploading: 0.000%|                                                                                                                 | 0/546206 elapsed<00:00 remaining<?
Uploading: 100.000%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 546206/546206 elapsed<00:02 remaining<00:00
Uploading model files (2 dirs, 1 files):  17%|███████████████▏                                                                           | 1/6 [00:03<00:01,  3.17it/s]
Uploading: 0.000%|                                                                                                                     | 0/56 elapsed<00:00 remaining<?
Uploading: 100.000%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 elapsed<00:01 remainin

Model created, explore it at https://c.app.hopsworks.ai:443/p/398/models/query_model/1


Model(name: 'query_model', version: 1)

In [23]:
# Define the input schema for the candidate_model based on item_df
candidate_model_input_schema = Schema(item_df)

# Define the output schema for the candidate_model, specifying the shape and type of the output
candidate_model_output_schema = Schema([{
    "name": "candidate_embedding",   # Name of the output feature
    "type": "float32",               # Data type of the output feature
    "shape": [EMB_DIM],              # Shape of the output feature
}])

# Combine the input and output schemas to create the overall model schema for the candidate_model
candidate_model_schema = ModelSchema(
    input_schema=candidate_model_input_schema,    # Input schema for the model
    output_schema=candidate_model_output_schema,  # Output schema for the model
)

# Sample a candidate example from the item DataFrame
candidate_example = item_df.sample().to_dict("records")

# Create a tensorflow model for the candidate_model in the Model Registry
mr_candidate_model = mr.tensorflow.create_model(
    name="candidate_model",                                        # Name of the model
    description="Model that generates candidate embeddings from video features",  # Description of the model
    input_example=candidate_example,                              # Example input for the model
    model_schema=candidate_model_schema,                          # Schema of the model
)

# Save the candidate_model to the Model Registry
mr_candidate_model.save("candidate_model")                        # Path to save the model

Uploading model files (2 dirs, 0 files):  17%|███████████████▏                                                                           | 1/6 [00:00<00:01,  3.05it/s]
Uploading: 0.000%|                                                                                                                 | 0/511735 elapsed<00:00 remaining<?
Uploading: 100.000%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 511735/511735 elapsed<00:02 remaining<00:00
Uploading model files (2 dirs, 1 files):  17%|███████████████▏                                                                           | 1/6 [00:03<00:01,  3.05it/s]
Uploading: 0.000%|                                                                                                                     | 0/55 elapsed<00:00 remaining<?
Uploading: 100.000%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 elapsed<00:01 remainin

Model created, explore it at https://c.app.hopsworks.ai:443/p/398/models/candidate_model/1


Model(name: 'candidate_model', version: 1)

---